# Side-Channel attack on ML-DSA

In [1]:
from tqdm.notebook import trange
import numpy as np
import time
import matplotlib.pyplot as plt
import random
import h5py
%matplotlib notebook

In [2]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_STM32F4'
CRYPTO_TARGET='NONE'
SS_VER='SS_VER_2_1' 

In [3]:
import chipwhisperer as cw

try:
    if not scope.connectStatus:
        scope.con()
except NameError:
    scope = cw.scope()

try:
    if SS_VER == "SS_VER_2_1":
        target_type = cw.targets.SimpleSerial2
    elif SS_VER == "SS_VER_2_0":
        raise OSError("SS_VER_2_0 is deprecated. Use SS_VER_2_1")
    else:
        target_type = cw.targets.SimpleSerial
except:
    SS_VER="SS_VER_1_1"
    target_type = cw.targets.SimpleSerial

try:
    target = cw.target(scope, target_type)
except:
    print("INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.")
    print("INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.")
    scope = cw.scope()
    target = cw.target(scope, target_type)

print("INFO: Found ChipWhisperer😍")

prog = cw.programmers.STM32FProgrammer

import time
time.sleep(0.05)
scope.default_setup()

def reset_target(scope):
    if PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
        scope.io.pdic = 'low'
        time.sleep(0.1)
        scope.io.pdic = 'high_z' #XMEGA doesn't like pdic driven high
        time.sleep(0.1) #xmega needs more startup time
    elif "neorv32" in PLATFORM.lower():
        raise IOError("Default iCE40 neorv32 build does not have external reset - reprogram device to reset")
    elif PLATFORM == "CW308_SAM4S" or PLATFORM == "CWHUSKY":
        scope.io.nrst = 'low'
        time.sleep(0.25)
        scope.io.nrst = 'high_z'
        time.sleep(0.25)
    else:  
        scope.io.nrst = 'low'
        time.sleep(0.05)
        scope.io.nrst = 'high_z'
        time.sleep(0.05)

INFO: Found ChipWhisperer😍
scope.gain.mode                          changed from low                       to high                     
scope.gain.gain                          changed from 0                         to 22                       
scope.gain.db                            changed from 15.0                      to 25.091743119266056       
scope.adc.samples                        changed from 131124                    to 5000                     
scope.adc.trig_count                     changed from 48880                     to 12538711                 
scope.clock.clkgen_freq                  changed from 0                         to 7363636.363636363        
scope.clock.adc_freq                     changed from 0                         to 29454545.454545453       
scope.io.tio1                            changed from serial_tx                 to serial_rx                
scope.io.tio2                            changed from serial_rx                 to serial_tx         

In [4]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd ../../firmware/mcu/simpleserial-mldsa
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 -j 

Building for platform CW308_STM32F4 with CRYPTO_TARGET=NONE
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
arm-none-eabi-gcc (15:10.3-2021.07-4) 10.3.1 20210621 (release)
Copyright (C) 2020 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

mkdir -p objdir-CW308_STM32F4 
.
Welcome to another exciting ChipWhisperer target build!!
.
.
.
.
.
.
.
Compiling:
Compiling:
.
Compiling:
Compiling:
Compiling:
Compiling:
.
Compiling:
Compiling:
.
-en     simpleserial-mldsa.c ...
-en     ntt.c ...
-en     packing.c ...
-en     polyvec.c ...
-en     poly.c ...
-en     rounding.c ...
-en     reduce.c ...
Compiling:
Compiling:
-en     sign.c ...
.
-en     symmetric-shake.c ...
-en     fips202.c ...
Compiling:
.
.
-en     randombytes.c ...
.
.
.
Compiling:
Compiling:
.
Compiling:
.
Compiling:
-en     .././simpleserial/simpleserial.c ...
Compiling:
-en     .././hal/hal.c

In file included from .././hal/chipwhisperer-fw-extra/stm32f4/stm32f4_hal.c:3:
.././hal/chipwhisperer-fw-extra/stm32f4/stm32f4_hal_lowlevel.h:108: warning: "STM32F415xx" redefined
  108 | #define STM32F415xx
      | 
<command-line>: note: this is the location of the previous definition
In file included from .././hal/chipwhisperer-fw-extra/stm32f4/stm32f4_hal_lowlevel.c:39:
.././hal/chipwhisperer-fw-extra/stm32f4/stm32f4_hal_lowlevel.h:108: warning: "STM32F415xx" redefined
  108 | #define STM32F415xx
      | 
<command-line>: note: this is the location of the previous definition


-e Done!
-e Done!
-e Done!
-e Done!
-e Done!
-e Done!
-e Done!
-e Done!
-e Done!
-e Done!
-e Done!
-e Done!
-e Done!
-e Done!
-e Done!
-e Done!
-e Done!
.
LINKING:
-en     simpleserial-mldsa-CW308_STM32F4.elf ...
-e Done!
.
.
.
.
.
Creating load file for Flash: simpleserial-mldsa-CW308_STM32F4.bin
Creating load file for Flash: simpleserial-mldsa-CW308_STM32F4.hex
arm-none-eabi-objcopy -O binary -R .eeprom -R .fuse -R .lock -R .signature simpleserial-mldsa-CW308_STM32F4.elf simpleserial-mldsa-CW308_STM32F4.bin
Creating Extended Listing: simpleserial-mldsa-CW308_STM32F4.lss
arm-none-eabi-objcopy -O ihex -R .eeprom -R .fuse -R .lock -R .signature simpleserial-mldsa-CW308_STM32F4.elf simpleserial-mldsa-CW308_STM32F4.hex
Creating load file for EEPROM: simpleserial-mldsa-CW308_STM32F4.eep
arm-none-eabi-objdump -h -S -z simpleserial-mldsa-CW308_STM32F4.elf > simpleserial-mldsa-CW308_STM32F4.lss
arm-none-eabi-objcopy -j .eeprom --set-section-flags=.eeprom="alloc,load" \
--change-section-lma .e

In [5]:
cw.program_target(scope, prog, "../../firmware/mcu/simpleserial-mldsa/simpleserial-mldsa-{}.hex".format(PLATFORM))

Detected known STMF32: STM32F40xxx/41xxx
Extended erase (0x44), this can take ten seconds or more
Attempting to program 22359 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 22359 bytes


In [6]:
def encode(slst):
    mask = 2+8*8380417
    buf = []
    for s in slst:
        s += mask
        for i in range(8):
            buf.append(s % 256)
            s = s // 256
    return bytearray(buf)

def decode(buf):
    slst = []
    mask = 2+8*8380417
    buf = list(buf)
    for j in range(len(buf)//8):
        n = 0
        for i in range(8):
            n = n * 256 + buf[j*8+7-i]
        slst.append(n - mask)
    return slst

In [7]:
scope.trigger.module = 'basic'
scope.adc.basic_mode = "rising_edge"
scope.trigger.triggers = "tio4"
scope.io.tio1 = "serial_rx"
scope.io.tio2 = "serial_tx"
scope.clock.clkgen_freq = 25e6 # sample rate = 100M/s
scope.io.hs2 = 'clkgen'
scope.clock.clkgen_src = 'system'
scope.clock.adc_mul = 4
scope.clock.reset_dcms()
reset_target(scope)
scope.adc.offset = 0
# scope.adc.samples = 1118000# 2500000
target.baud *= scope.clock.clkgen_freq / 7.37e6

In [24]:
# ldr s
with h5py.File("D:\\Code_Repo\\Redundant_Vulnerabilities_in_Rejection_Sampling\\examination\\TA\\Trace\\profile_data_s1hat.h5", "r") as f:
    profile_data = f["data"][:]
    
with h5py.File("D:\\Code_Repo\\Redundant_Vulnerabilities_in_Rejection_Sampling\\examination\\TA\\Trace\\test_data_s1hat.h5", "r") as f:
    test_data = f["data"][:]
print(profile_data.shape)
print(test_data.shape)

(33, 500)
(5000,)


In [25]:
scope.adc.samples = 500
Q = 8380417

In [26]:
print(profile_data.shape)

(33, 500)


In [27]:
def Trace_capture_profile_s(nb_cls, nb_cls_trc, profile_data):
    idx = 0
    trace_array = np.zeros([nb_cls*nb_cls_trc, scope.adc.samples])
    for i in trange(nb_cls, desc = 'HW'):
        for j in trange(nb_cls_trc, desc=f'Capturing traces of HW {i}'):
            scope.arm()
            c = random.randint(-1-4*Q, 1+4*Q)
            text = encode([c, int(profile_data[i][j])])
            target.simpleserial_write(0x01, text)
            ret = scope.capture()
            if ret:
                print("Target timed out!")
                continue
            response = target.simpleserial_read('r', 16)
            trace_array[idx] += np.array(scope.get_last_trace())
            idx += 1
    return trace_array

def Trace_capture_test_s(nb_trc, test_data):
    idx = 0
    test_trace = np.zeros([nb_trc, scope.adc.samples])
    for i in trange(nb_trc, desc='Capturing traces'):
        scope.arm()
        c = random.randint(-1-4*Q, 1+4*Q)
        text = encode([c, int(test_data[i])])
        target.simpleserial_write('p', text)
        ret = scope.capture()
        if ret:
            print("Target timed out!")
            continue
        response = target.simpleserial_read('r', 16)
        test_trace[idx] += np.array(scope.get_last_trace())
        idx += 1
    return test_trace

In [ ]:
for i in range(6):
    nb_cls, nb_cls_trc = profile_data.shape
    profile_trace = Trace_capture_profile_s(nb_cls, nb_cls_trc, profile_data)
    nb_trc = test_data.shape[0]
    test_trace = Trace_capture_test_s(nb_trc, test_data)
    trace_path = "D:\\Code_Repo\\Redundant_Vulnerabilities_in_Rejection_Sampling\\examination\\TA\\Trace\\"
    with h5py.File(trace_path + f"profile_trace_s_{i}.h5", "w") as f:
        f.create_dataset("trace", data=profile_trace)
    with h5py.File(trace_path + f"test_trace_s_{i}.h5", "w") as f:
        f.create_dataset("trace", data=test_trace)

HW:   0%|          | 0/33 [00:00<?, ?it/s]

Capturing traces of HW 0:   0%|          | 0/500 [00:00<?, ?it/s]

Capturing traces of HW 1:   0%|          | 0/500 [00:00<?, ?it/s]

Capturing traces of HW 2:   0%|          | 0/500 [00:00<?, ?it/s]

Capturing traces of HW 3:   0%|          | 0/500 [00:00<?, ?it/s]

...

Capturing traces of HW 29:   0%|          | 0/500 [00:00<?, ?it/s]

Capturing traces of HW 30:   0%|          | 0/500 [00:00<?, ?it/s]

Capturing traces of HW 31:   0%|          | 0/500 [00:00<?, ?it/s]

Capturing traces of HW 32:   0%|          | 0/500 [00:00<?, ?it/s]

Capturing traces:   0%|          | 0/5000 [00:00<?, ?it/s]

In [14]:
# str x
with h5py.File("D:\\Code_Repo\\Redundant_Vulnerabilities_in_Rejection_Sampling\\examination\\TA\\Trace\\profile_data_xhat.h5", "r") as f:
    profile_data_s = f["s1_data"][:]
    profile_data_c = f["c_data"][:]
with h5py.File("D:\\Code_Repo\\Redundant_Vulnerabilities_in_Rejection_Sampling\\examination\\TA\\Trace\\test_data_xhat.h5", "r") as f:
    test_data_s = f["s1_data"][:]
    test_data_c = f["c_data"][:]
print(profile_data_s.shape)
print(test_data_s.shape)

(33, 500)
(5000,)


In [15]:
scope.adc.samples = 500# 30000

In [16]:
def Trace_capture_profile_x(nb_cls, nb_cls_trc, profile_data_c, profile_data_s):
    trace_array = np.zeros([nb_cls * nb_cls_trc, scope.adc.samples])
    idx = 0
    for i in trange(nb_cls, desc = 'HW:'):
        for j in trange(nb_cls_trc, desc=f'Capturing traces of HW {i}'):
            scope.arm()
            text = encode([int(profile_data_c[i][j]), int(profile_data_s[i][j])])
            target.simpleserial_write('p', text)
            ret = scope.capture()
            if ret:
                print("Target timed out!")
                continue
            response = target.simpleserial_read('r', 16)
            trace_array[idx] += np.array(scope.get_last_trace())
            idx += 1
    return trace_array
    
def Trace_capture_test_x(nb_trc, test_data_c, test_data_s):
    test_trace = np.zeros([nb_trc, scope.adc.samples])
    idx = 0
    for i in trange(nb_trc, desc='Capturing traces'):
        scope.arm()
        text = encode([int(test_data_c[i]), int(test_data_s[i])])
        target.simpleserial_write('p', text)
        ret = scope.capture()
        if ret:
            print("Target timed out!")
            continue
        response = target.simpleserial_read('r', 16)
        test_trace[idx] += np.array(scope.get_last_trace())
        idx += 1
    return test_trace

In [17]:
for i in range(4):
    nb_cls, nb_cls_trc = profile_data_s.shape
    profile_trace = Trace_capture_profile_x(nb_cls, nb_cls_trc, profile_data_c, profile_data_s)
    nb_trc = test_data_s.shape[0]
    test_trace = Trace_capture_test_x(nb_trc, test_data_c, test_data_s)
    trace_path = "D:\\Code_Repo\\Redundant_Vulnerabilities_in_Rejection_Sampling\\examination\\TA\\Trace\\"
    with h5py.File(trace_path + f"profile_trace_x_{i}.h5", "w") as f:
        f.create_dataset("trace", data=profile_trace)
    with h5py.File(trace_path + f"test_trace_x_{i}.h5", "w") as f:
        f.create_dataset("trace", data=test_trace)

HW::   0%|          | 0/33 [00:00<?, ?it/s]

Capturing traces of HW 0:   0%|          | 0/500 [00:00<?, ?it/s]

Capturing traces of HW 1:   0%|          | 0/500 [00:00<?, ?it/s]

Capturing traces of HW 2:   0%|          | 0/500 [00:00<?, ?it/s]

Capturing traces of HW 3:   0%|          | 0/500 [00:00<?, ?it/s]

...

Capturing traces of HW 29:   0%|          | 0/500 [00:00<?, ?it/s]

Capturing traces of HW 30:   0%|          | 0/500 [00:00<?, ?it/s]

Capturing traces of HW 31:   0%|          | 0/500 [00:00<?, ?it/s]

Capturing traces of HW 32:   0%|          | 0/500 [00:00<?, ?it/s]

Capturing traces:   0%|          | 0/5000 [00:00<?, ?it/s]

In [22]:
# Attack

In [18]:
with h5py.File("D:\\Code_Repo\\Redundant_Vulnerabilities_in_Rejection_Sampling\\examination\\TA\\Trace\\attack_data_cs1.h5", "r") as f:
    s1hat = f["s1hat"][:]
    chat = f["chat"][:]
print(s1hat.shape)
print(chat.shape)

(4, 256)
(1000, 256)


In [21]:
def Trace_capture_attack(nb_trc, chat, s1):
    test_trace = np.zeros([256, nb_trc, scope.adc.samples])
    for idx in trange(256):
        cnt = 0
        for i in trange(nb_trc, desc='Capturing traces'):
            scope.arm()
            text = encode([int(chat[i][idx]), int(s1[idx])])
            target.simpleserial_write('p', text)
            ret = scope.capture()
            if ret:
                print("Target timed out!")
                continue
            response = target.simpleserial_read('r', 16)
            test_trace[idx][cnt] += np.array(scope.get_last_trace())
            cnt += 1
    return test_trace

In [23]:
for q in range(4):
    attack_trace = Trace_capture_attack(1000, chat, s1hat[q])
    with h5py.File(f"D:\\Code_Repo\\Redundant_Vulnerabilities_in_Rejection_Sampling\\examination\\TA\\Trace\\attack_trace_cs1_{q}.h5", "w") as f:
        for idx in range(256):
            g = f.create_group(f"{idx}")
            g.create_dataset("trace", data=attack_trace[idx])

  0%|          | 0/256 [00:00<?, ?it/s]

Capturing traces:   0%|          | 0/1000 [00:00<?, ?it/s]

Capturing traces:   0%|          | 0/1000 [00:00<?, ?it/s]

Capturing traces:   0%|          | 0/1000 [00:00<?, ?it/s]

Capturing traces:   0%|          | 0/1000 [00:00<?, ?it/s]

...

Capturing traces:   0%|          | 0/1000 [00:00<?, ?it/s]

Capturing traces:   0%|          | 0/1000 [00:00<?, ?it/s]

Capturing traces:   0%|          | 0/1000 [00:00<?, ?it/s]

Capturing traces:   0%|          | 0/1000 [00:00<?, ?it/s]

Capturing traces:   0%|          | 0/1000 [00:00<?, ?it/s]

In [14]:
scope.dis()
target.dis()